<a href="https://colab.research.google.com/github/lpoggetto/trabalho_final_pyspark/blob/main/trabalho_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalando spark
!pip install pyspark==3.3.1
# importando pacotes necessarios
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
# iniciando sessao do spark
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Analise de jogos internacionais de futebo') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [4]:
# checando sessao
spark

In [6]:
# Lendo o csv com pandas
pandas_df = pd.read_csv('/content/dados_trab.csv')

# convertendo o arquivo para PySpark
df = spark.createDataFrame(pandas_df.to_dict(orient = 'records'))
df

DataFrame[away_score: bigint, away_team: string, city: string, country: string, date: string, home_score: bigint, home_team: string, neutral: boolean, tournament: string]

In [7]:
# visao inicial do dataset
df.show()

+----------+---------+---------+-------------------+----------+----------+----------------+-------+----------+
|away_score|away_team|     city|            country|      date|home_score|       home_team|neutral|tournament|
+----------+---------+---------+-------------------+----------+----------+----------------+-------+----------+
|         0|  England|  Glasgow|           Scotland|1872-11-30|         0|        Scotland|  false|  Friendly|
|         2| Scotland|   London|            England|1873-03-08|         4|         England|  false|  Friendly|
|         1|  England|  Glasgow|           Scotland|1874-03-07|         2|        Scotland|  false|  Friendly|
|         2| Scotland|   London|            England|1875-03-06|         2|         England|  false|  Friendly|
|         0|  England|  Glasgow|           Scotland|1876-03-04|         3|        Scotland|  false|  Friendly|
|         0|    Wales|  Glasgow|           Scotland|1876-03-25|         4|        Scotland|  false|  Friendly|
|

In [8]:
# printando o schema da tabela
df.printSchema()

root
 |-- away_score: long (nullable = true)
 |-- away_team: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date: string (nullable = true)
 |-- home_score: long (nullable = true)
 |-- home_team: string (nullable = true)
 |-- neutral: boolean (nullable = true)
 |-- tournament: string (nullable = true)



### 1 - Quantos registros existem na base?

R: A base conta com 40839 registros



In [10]:
df.count()

40839

### 2 - Quantas equipes únicas mandantes existem na base?

R: 309 equipes mandantes unicas

In [12]:
df.select('home_team').distinct().count()

309

### 3 - Quantas vezes as equipes mandantes saíram vitoriosas?

R: As equipes mandantes foram vitoriosas 19.864 vezes ou seja, em 48% das partidas

In [13]:
df.select('home_team', 'home_score', 'away_score').filter(df.home_score > df.away_score).count()

19864

### 4 - Quantas vezes as equipes visitantes saíram vitoriosas?

R: As equipes visitantes foram vitoriosas 11.544 vezes ou seja, em 28% das partidas

In [14]:
df.select('home_team', 'home_score', 'away_score').filter(df.home_score < df.away_score).count()

11544

### 5 - Quantas partidas resultaram em empate?

R: 9431 partidas resultaram em um empate, ou seja, 23% das partidas.

In [16]:
df.select('home_team', 'home_score', 'away_score').filter(df.home_score == df.away_score).count()

9431

### 6 - Quantas partidas foram realizadas em cada país?

In [27]:
print('Listando todos as partidas por País')
df.select('country') \
  .groupBy('country') \
  .count() \
  .orderBy('count', ascending = False) \
  .show(n = 100000, truncate = False)

Listando todos as partidas por País
+--------------------------------+-----+
|country                         |count|
+--------------------------------+-----+
|United States                   |1144 |
|France                          |801  |
|England                         |687  |
|Malaysia                        |644  |
|Sweden                          |637  |
|Germany                         |581  |
|Brazil                          |529  |
|Spain                           |517  |
|Thailand                        |483  |
|Italy                           |480  |
|Switzerland                     |477  |
|Austria                         |475  |
|United Arab Emirates            |472  |
|South Africa                    |470  |
|Qatar                           |467  |
|South Korea                     |453  |
|Argentina                       |449  |
|Hungary                         |431  |
|Chile                           |405  |
|Belgium                         |396  |
|Mexico              

### 7 - Qual país teve mais partidas?

R: O País com a maior qtd. de partidas é os Estados Unidos com 1144 partidas.

In [28]:
df.select('country') \
  .groupBy('country') \
  .count() \
  .orderBy('count', ascending = False) \
  .limit(1) \
  .show()

+-------------+-----+
|      country|count|
+-------------+-----+
|United States| 1144|
+-------------+-----+



### 8 - Qual a partida com maior número de gols?

R: A partida com o maior número de gols foi um jogo de Austrália vs Samoa Americana com um total de 31 gols

In [38]:
# Calculando soma dos gols para as partidas
soma_de_gols = df.withColumn("score_sum", df["home_score"] + df["away_score"])

# Select relevant columns and order by the sum in descending order
result = soma_de_gols.select('home_team', 'away_team', 'home_score', 'away_score', 'score_sum') \
    .orderBy('score_sum', ascending=False) \
    .limit(1)

# Show the result
result.show()

+---------+--------------+----------+----------+---------+
|home_team|     away_team|home_score|away_score|score_sum|
+---------+--------------+----------+----------+---------+
|Australia|American Samoa|        31|         0|       31|
+---------+--------------+----------+----------+---------+



### 9 - Qual a maior goleada?


R: A maior goleada foi também com o maior número de gols foi um jogo de Austrália vs Samoa Americana em que a Austrália ganhou de 31 a 0.

In [34]:
# Calculando a a maior diferenca entre gols
df_with_diff = df.withColumn("score_diff", F.abs(df["home_score"] - df["away_score"]))

# Encontrando a linha com a maior diferenca
max_diff_row = df_with_diff.orderBy(F.desc("score_diff")).first()

# extraindo a maior goleada
max_diff = max_diff_row["score_diff"]

print("Maior diferença de gols:", max_diff)

Maior diferença de gols: 31


### 10 - Quantos jogos ocorreram no Brasil?

R: Ocorreram 529 jogos no Brasil.

In [40]:
df.filter(df.country == 'Brazil').count()

529